In [ ]:
# Load generated recipes files
import pandas as pd
# Load the data
manual = pd.read_json('../data/generated_recipes_manual_review.json', dtype={'correct': 'boolean'})
rag = pd.read_json('../data/generated_recipes_rag_review.json', dtype={'correct': 'boolean'})
moderator = pd.read_json('../data/generated_recipes_moderator_review.json', dtype={'correct': 'boolean'})
combined = pd.read_json('../data/generated_recipes_combined_review.json', dtype={'correct': 'boolean'})
rag.rename(columns={'correct': 'correct_rag'}, inplace=True)
moderator.rename(columns={'correct': 'correct_moderator'}, inplace=True)
combined.rename(columns={'correct': 'correct_combined'}, inplace=True)
totalset = pd.merge(manual, rag[['medische_vraag', 'correct_rag']], on='medische_vraag', how='left')
totalset = pd.merge(totalset, moderator[['medische_vraag', 'correct_moderator']], on='medische_vraag', how='left')
totalset = pd.merge(totalset, combined[['medische_vraag', 'correct_combined']], on='medische_vraag', how='left')
# Totaal aantal vragen
totaal_vragen = totalset.shape[0]
# Aantal correcte antwoorden
correct_counts = totalset[['correct', 'correct_rag', 'correct_moderator', 'correct_combined']].sum()
# True Positives en True Negatives berekeningen
true_positives_rag = totalset[(totalset['correct'] == True) & (totalset['correct_rag'] == True)].shape[0]
true_negatives_rag = totalset[(totalset['correct'] == False) & (totalset['correct_rag'] == False)].shape[0]
false_positives_rag = totalset[(totalset['correct'] == False) & (totalset['correct_rag'] == True)].shape[0]
false_negatives_rag = totalset[(totalset['correct'] == True) & (totalset['correct_rag'] == False)].shape[0]
true_positives_moderator = totalset[(totalset['correct'] == True) & (totalset['correct_moderator'] == True)].shape[0]
true_negatives_moderator = totalset[(totalset['correct'] == False) & (totalset['correct_moderator'] == False)].shape[0]
false_positives_moderator = totalset[(totalset['correct'] == False) & (totalset['correct_moderator'] == True)].shape[0]
false_negatives_moderator = totalset[(totalset['correct'] == True) & (totalset['correct_moderator'] == False)].shape[0]
true_positives_combined = totalset[(totalset['correct'] == True) & (totalset['correct_combined'] == True)].shape[0]
true_negatives_combined = totalset[(totalset['correct'] == False) & (totalset['correct_combined'] == False)].shape[0]
false_positives_combined = totalset[(totalset['correct'] == False) & (totalset['correct_combined'] == True)].shape[0]
false_negatives_combined = totalset[(totalset['correct'] == True) & (totalset['correct_combined'] == False)].shape[0]
# Totaalberekeningen
totaal_rag = true_positives_rag + true_negatives_rag + false_positives_rag + false_negatives_rag
totaal_moderator = true_positives_moderator + true_negatives_moderator + false_positives_moderator + false_negatives_moderator
totaal_combined = true_positives_combined + true_negatives_combined + false_positives_combined + false_negatives_combined
# Print resultaten
print("True Positives, True Negatives, False Positives, False Negatives en Totalen:")
print(f"RAG -> TP: {true_positives_rag}, TN: {true_negatives_rag}, FP: {false_positives_rag}, FN: {false_negatives_rag}, Total: {totaal_rag}")
print(f"Moderator -> TP: {true_positives_moderator}, TN: {true_negatives_moderator}, FP: {false_positives_moderator}, FN: {false_negatives_moderator}, Total: {totaal_moderator}")
print(f"Combined -> TP: {true_positives_combined}, TN: {true_negatives_combined}, FP: {false_positives_combined}, FN: {false_negatives_combined}, Total: {totaal_combined}")


In [ ]:
print# Count the number of correct answers
correct_counts = totalset[['correct', 'correct_rag', 'correct_moderator', 'correct_combined']].sum()
# Print the counts
print("Number of correct answers:")
print(correct_counts)

false_positives_rag = totalset[(totalset['correct'] == False) & (totalset['correct_rag'] == True)][
    'medische_vraag'].count()
false_positives_moderator = totalset[(totalset['correct'] == False) & (totalset['correct_moderator'] == True)][
    'medische_vraag'].count()
false_positives_combined = totalset[(totalset['correct'] == False) & (totalset['correct_combined'] == True)][
    'medische_vraag'].count()

print(f"Number of false positives rag: {false_positives_rag}")
print(f"Number of false positives moderator: {false_positives_moderator}")
print(f"Number of false positives combined: {false_positives_combined}")

false_negatives_rag = totalset[(totalset['correct'] == True) & (totalset['correct_rag'] == False)][
    'medische_vraag'].count()
false_negatives_moderator = totalset[(totalset['correct'] == True) & (totalset['correct_moderator'] == False)][
    'medische_vraag'].count()
false_negatives_combined = totalset[(totalset['correct'] == True) & (totalset['correct_combined'] == False)][
    'medische_vraag'].count()
print(f"Number of false negatives rag: {false_negatives_rag}")
print(f"Number of false negatives moderator: {false_negatives_moderator}")
print(f"Number of false negatives combined: {false_negatives_combined}")

precision_rag = correct_counts['correct_rag'] / (correct_counts['correct_rag'] + false_positives_rag) if (
                                                                                                                 correct_counts[
                                                                                                                     'correct_rag'] + false_positives_rag) > 0 else 0
recall_rag = correct_counts['correct_rag'] / (correct_counts['correct_rag'] + false_negatives_rag) if (correct_counts[
                                                                                                           'correct_rag'] + false_negatives_rag) > 0 else 0
precision_moderator = correct_counts['correct_moderator'] / (
        correct_counts['correct_moderator'] + false_positives_moderator) if (correct_counts[
                                                                                 'correct_moderator'] + false_positives_moderator) > 0 else 0
recall_moderator = correct_counts['correct_moderator'] / (
        correct_counts['correct_moderator'] + false_negatives_moderator) if (correct_counts[
                                                                                 'correct_moderator'] + false_negatives_moderator) > 0 else 0
precision_combined = correct_counts['correct_combined'] / (
        correct_counts['correct_combined'] + false_positives_combined) if (correct_counts[
                                                                               'correct_combined'] + false_positives_combined) > 0 else 0
recall_combined = correct_counts['correct_combined'] / (
        correct_counts['correct_combined'] + false_negatives_combined) if (correct_counts[
                                                                               'correct_combined'] + false_negatives_combined) > 0 else 0

print(f"Precision rag: {precision_rag:.2f} Recall rag: {recall_rag:.2f}")
print(f"Precision moderator: {precision_moderator:.2f} Recall moderator: {recall_moderator:.2f}")
print(f"Precision combined: {precision_combined:.2f} Recall combined: {recall_combined:.2f}")

print("#########################")
print("#########################")
print("#########################")
print("#########################")

# False positives: Laat 'correct', 'correct_rag' en 'medische_vraag' zien
false_positives_rag = totalset[
    (totalset['correct'] == False) & (totalset['correct_rag'] == True)
    ][['correct', 'correct_rag', 'medische_vraag']]

# False negatives: Laat 'correct', 'correct_rag' en 'medische_vraag' zien
false_negatives_rag = totalset[
    (totalset['correct'] == True) & (totalset['correct_rag'] == False)
    ][['correct', 'correct_rag', 'medische_vraag']]

# Correcte voorspellingen: Waar correct en correct_rag overeenkomen
correct_predictions_rag = totalset[
    totalset['correct'] == totalset['correct_rag']
    ][['correct', 'correct_rag', 'medische_vraag']]

# Print de waarden voor false positives
print("False positives van RAG (met correcte waarde en voorspelling):")
print(false_positives_rag)

# Print de waarden voor false negatives
print("\nFalse negatives van RAG (met correcte waarde en voorspelling):")
print(false_negatives_rag)

# Print de waarden voor correcte voorspellingen
print("\nCorrecte voorspellingen van RAG:")
print(correct_predictions_rag)

# False positives: Laat 'correct', 'correct_moderator' en 'medische_vraag' zien
false_positives_moderator = totalset[
    (totalset['correct'] == False) & (totalset['correct_moderator'] == True)
    ][['correct', 'correct_moderator', 'medische_vraag']]

false_positives_combined = totalset[
    (totalset['correct'] == False) & (totalset['correct_combined'] == True)
    ][['correct', 'correct_combined', 'medische_vraag']]

# False negatives: Laat 'correct', 'correct_moderator' en 'medische_vraag' zien
false_negatives_moderator = totalset[
    (totalset['correct'] == True) & (totalset['correct_moderator'] == False)
    ][['correct', 'correct_moderator', 'medische_vraag']]

false_negatives_combined = totalset[
    (totalset['correct'] == True) & (totalset['correct_combined'] == False)
    ][['correct', 'correct_combined', 'medische_vraag']]

# Correcte voorspellingen: Waar correct en correct_moderator overeenkomen
correct_predictions_moderator = totalset[
    totalset['correct'] == totalset['correct_moderator']
    ][['correct', 'correct_moderator', 'medische_vraag']]

# Correcte voorspellingen: Waar correct en correct_combined overeenkomen
correct_predictions_combined = totalset[
    totalset['correct'] == totalset['correct_combined']
    ][['correct', 'correct_combined', 'medische_vraag']]

# Print resultaten
print("False positives van moderator:")
print(false_positives_moderator)

print("\nFalse negatives van moderator:")
print(false_negatives_moderator)

print("\nCorrecte voorspellingen van moderator:")
print(correct_predictions_moderator)

print("\n#########################")

print("\nFalse positives van combined:")
print(false_positives_combined)

print("\nFalse negatives van combined:")
print(false_negatives_combined)

print("\nCorrecte voorspellingen van combined:")
print(correct_predictions_combined)

print("#########################")
print("#########################")
print("FALSE POSITIVES")
print("#########################")

# False positives: 'medische_vraag' voor beide datasets
false_positives_rag_set = set(
    totalset[
        (totalset['correct'] == False) & (totalset['correct_rag'] == True)
        ]['medische_vraag']
)

false_positives_moderator_set = set(
    totalset[
        (totalset['correct'] == False) & (totalset['correct_moderator'] == True)
        ]['medische_vraag']
)

# Overlapping tussen false positives van RAG en moderator
overlapping_false_positives = false_positives_rag_set & false_positives_moderator_set
# False positives in RAG maar niet in moderator
false_positives_rag_not_moderator = false_positives_rag_set - false_positives_moderator_set
# False positives in moderator maar niet in RAG
false_positives_moderator_not_rag = false_positives_moderator_set - false_positives_rag_set

# Aantallen berekenen
aantal_overlapping = len(overlapping_false_positives)
aantal_rag_not_moderator = len(false_positives_rag_not_moderator)
aantal_moderator_not_rag = len(false_positives_moderator_not_rag)

# Resultaten printen
print(f"Aantal overlapping false positives tussen RAG en moderator: {aantal_overlapping}")
print(f"Aantal false positives uniek voor RAG (niet in moderator): {aantal_rag_not_moderator}")
print(f"Aantal false positives uniek voor moderator (niet in RAG): {aantal_moderator_not_rag}")

# Optioneel: de overlappende of niet-overlappende waarden printen
print("\nOverlapping false positives:")
print(overlapping_false_positives)

print("\nFalse positives in RAG maar niet in moderator:")
print(false_positives_rag_not_moderator)

print("\nFalse positives in moderator maar niet in RAG:")
print(false_positives_moderator_not_rag)

print("#########################")
print("#########################")
print("FALSE NEGATIVES")
print("#########################")

# False negatives: 'medische_vraag' voor beide datasets
false_negatives_rag_set = set(
    totalset[
        (totalset['correct'] == True) & (totalset['correct_rag'] == False)
        ]['medische_vraag']
)
false_negatives_moderator_set = set(
    totalset[
        (totalset['correct'] == True) & (totalset['correct_moderator'] == False)
        ]['medische_vraag']
)

# Overlapping tussen false negatives van RAG en moderator
overlapping_false_negatives = false_negatives_rag_set & false_negatives_moderator_set
# False negatives in RAG maar niet in moderator
false_negatives_rag_not_moderator = false_negatives_rag_set - false_negatives_moderator_set
# False negatives in moderator maar niet in RAG
false_negatives_moderator_not_rag = false_negatives_moderator_set - false_negatives_rag_set

# Aantallen berekenen
aantal_overlapping_fn = len(overlapping_false_negatives)
aantal_rag_not_moderator_fn = len(false_negatives_rag_not_moderator)
aantal_moderator_not_rag_fn = len(false_negatives_moderator_not_rag)

# Resultaten printen
print(f"Aantal overlapping false negatives tussen RAG en moderator: {aantal_overlapping_fn}")
print(f"Aantal false negatives uniek voor RAG (niet in moderator): {aantal_rag_not_moderator_fn}")
print(f"Aantal false negatives uniek voor moderator (niet in RAG): {aantal_moderator_not_rag_fn}")

# Optioneel: de overlappende of niet-overlappende waarden printen
print("\nOverlapping false negatives:")
print(overlapping_false_negatives)

print("\nFalse negatives in RAG maar niet in moderator:")
print(false_negatives_rag_not_moderator)

print("\nFalse negatives in moderator maar niet in RAG:")
print(false_negatives_moderator_not_rag)


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Data inladen (aanpassen aan je bestandslocatie)
manual = pd.read_json('../data/generated_recipes_manual_review.json', dtype={'correct': 'boolean'})
rag = pd.read_json('../data/generated_recipes_rag_review.json', dtype={'correct': 'boolean'})
moderator = pd.read_json('../data/generated_recipes_moderator_review.json', dtype={'correct': 'boolean'})
combined = pd.read_json('../data/generated_recipes_combined_review.json', dtype={'correct': 'boolean'})

manual['correct'] = manual['correct'].fillna(False)
rag.rename(columns={'correct': 'correct_rag'}, inplace=True)
moderator.rename(columns={'correct': 'correct_moderator'}, inplace=True)
combined.rename(columns={'correct': 'correct_combined'}, inplace=True)

# Combineer alle datasets
totalset = pd.merge(manual, rag[['medische_vraag', 'correct_rag']], on='medische_vraag', how='left')
totalset = pd.merge(totalset, moderator[['medische_vraag', 'correct_moderator']], on='medische_vraag', how='left')
totalset = pd.merge(totalset, combined[['medische_vraag', 'correct_combined']], on='medische_vraag', how='left')


# Functie om false positives, false negatives, unique values en overlap te berekenen
def compare_predictions(method1, method2, label1, label2):
    print(f"\nVergelijking: {label1} vs {label2}")

    # False Positives en False Negatives
    false_positives_1 = set(totalset[
                                (totalset['correct'] == False) & (totalset[f'correct_{method1}'] == True)
                                ]['medische_vraag'])

    false_negatives_1 = set(totalset[
                                (totalset['correct'] == True) & (totalset[f'correct_{method1}'] == False)
                                ]['medische_vraag'])

    false_positives_2 = set(totalset[
                                (totalset['correct'] == False) & (totalset[f'correct_{method2}'] == True)
                                ]['medische_vraag'])

    false_negatives_2 = set(totalset[
                                (totalset['correct'] == True) & (totalset[f'correct_{method2}'] == False)
                                ]['medische_vraag'])

    # Overlap en unieke fouten
    overlapping_fp = false_positives_1 & false_positives_2
    unique_fp_1 = false_positives_1 - false_positives_2
    unique_fp_2 = false_positives_2 - false_positives_1

    overlapping_fn = false_negatives_1 & false_negatives_2
    unique_fn_1 = false_negatives_1 - false_negatives_2
    unique_fn_2 = false_negatives_2 - false_negatives_1

    # Resultaten printen
    print(f"Aantal False Positives in {label1}: {len(false_positives_1)}")
    print(f"Aantal False Positives in {label2}: {len(false_positives_2)}")
    print(f"Overlappende False Positives: {len(overlapping_fp)}")
    print(f"Unieke False Positives in {label1}: {len(unique_fp_1)}")
    print(f"Unieke False Positives in {label2}: {len(unique_fp_2)}")
    print(f"Niet overlappende False Positives in {label1}: {unique_fp_1}")
    print(f"Niet overlappende False Positives in {label2}: {unique_fp_2}")

    print(f"\nAantal False Negatives in {label1}: {len(false_negatives_1)}")
    print(f"Aantal False Negatives in {label2}: {len(false_negatives_2)}")
    print(f"Overlappende False Negatives: {len(overlapping_fn)}")
    print(f"Unieke False Negatives in {label1}: {len(unique_fn_1)}")
    print(f"Unieke False Negatives in {label2}: {len(unique_fn_2)}")
    print(f"Niet overlappende False Negatives in {label1}: {unique_fn_1}")
    print(f"Niet overlappende False Negatives in {label2}: {unique_fn_2}\n")

    # Confusiematrix voor Methode 1
    true_positive_1 = len(totalset[
                              (totalset['correct'] == True) & (totalset[f'correct_{method1}'] == True)
                              ])
    true_negative_1 = len(totalset[
                              (totalset['correct'] == False) & (totalset[f'correct_{method1}'] == False)
                              ])
    false_positive_1 = len(false_positives_1)
    false_negative_1 = len(false_negatives_1)

    confusion_matrix_1 = [
        [true_positive_1, false_negative_1],
        [false_positive_1, true_negative_1]
    ]

    sns.heatmap(confusion_matrix_1, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Correct', 'Incorrect'], yticklabels=['Correct', 'Incorrect'])
    plt.title(f"Confusiematrix {label1}")
    plt.show()

    # Confusiematrix voor Methode 2
    true_positive_2 = len(totalset[
                              (totalset['correct'] == True) & (totalset[f'correct_{method2}'] == True)
                              ])
    true_negative_2 = len(totalset[
                              (totalset['correct'] == False) & (totalset[f'correct_{method2}'] == False)
                              ])
    false_positive_2 = len(false_positives_2)
    false_negative_2 = len(false_negatives_2)

    confusion_matrix_2 = [
        [true_positive_2, false_negative_2],
        [false_positive_2, true_negative_2]
    ]

    sns.heatmap(confusion_matrix_2, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Correct', 'Incorrect'], yticklabels=['Correct', 'Incorrect'])
    plt.title(f"Confusiematrix {label2}")
    plt.show()


# Vergelijkingen maken
compare_predictions('rag', 'moderator', 'RAG', 'MODERATOR')
compare_predictions('rag', 'combined', 'RAG', 'COMBINED')
compare_predictions('moderator', 'combined', 'MODERATOR', 'COMBINED')

In [ ]:
from matplotlib_venn import venn3
import matplotlib.pyplot as plt

# Sets for false positives and false negatives
fp_rag_set = set(totalset[(totalset['correct'] == False) & (totalset['correct_rag'] == True)]['medische_vraag'])
fp_moderator_set = set(
    totalset[(totalset['correct'] == False) & (totalset['correct_moderator'] == True)]['medische_vraag'])
fp_combined_set = set(
    totalset[(totalset['correct'] == False) & (totalset['correct_combined'] == True)]['medische_vraag'])

fn_rag_set = set(totalset[(totalset['correct'] == True) & (totalset['correct_rag'] == False)]['medische_vraag'])
fn_moderator_set = set(
    totalset[(totalset['correct'] == True) & (totalset['correct_moderator'] == False)]['medische_vraag'])
fn_combined_set = set(
    totalset[(totalset['correct'] == True) & (totalset['correct_combined'] == False)]['medische_vraag'])

# Plot Venn Diagram for False Positives
plt.figure(figsize=(10, 5))
plt.title("Venn Diagram of False Positives")
venn3([fp_rag_set, fp_moderator_set, fp_combined_set], set_labels=("RAG", "Moderator", "Combined"))
plt.show()

# Plot Venn Diagram for False Negatives
plt.figure(figsize=(10, 5))
plt.title("Venn Diagram of False Negatives")
venn3([fn_rag_set, fn_moderator_set, fn_combined_set], set_labels=("RAG", "Moderator", "Combined"))
plt.show()

# Summary Table
fp_overlap_summary = {
    "FP unique to RAG": len(fp_rag_set - fp_moderator_set - fp_combined_set),
    "FP unique to Moderator": len(fp_moderator_set - fp_rag_set - fp_combined_set),
    "FP unique to Combined": len(fp_combined_set - fp_rag_set - fp_moderator_set),
    "FP overlap (all)": len(fp_rag_set & fp_moderator_set & fp_combined_set)
}

fn_overlap_summary = {
    "FN unique to RAG": len(fn_rag_set - fn_moderator_set - fn_combined_set),
    "FN unique to Moderator": len(fn_moderator_set - fn_rag_set - fn_combined_set),
    "FN unique to Combined": len(fn_combined_set - fn_rag_set - fn_moderator_set),
    "FN overlap (all)": len(fn_rag_set & fn_moderator_set & fn_combined_set)
}

print("False Positives Overlap Summary:")
print(fp_overlap_summary)
print("\nFalse Negatives Overlap Summary:")
print(fn_overlap_summary)

#####################################################################

correct_predictions = totalset[['correct_rag', 'correct_moderator', 'correct_combined']].sum()
incorrect_predictions = totalset[['correct_rag', 'correct_moderator', 'correct_combined']].count() - correct_predictions

# Bar chart
labels = ['RAG', 'Moderator', 'Combined']
x = range(len(labels))

plt.bar(x, correct_predictions, label="Correct", color='g')
plt.bar(x, incorrect_predictions, bottom=correct_predictions, label="Incorrect", color='r')

plt.xticks(x, labels)
plt.title("Correct vs Incorrect Predictions")
plt.ylabel("Count")
plt.legend()
plt.show()